In [ ]:
import modin.pandas as pd
import numpy as np
import snowflake.snowpark.modin.plugin
import random
from faker import Faker
from snowflake.snowpark.context import get_active_session

### Helper Functions: Snowflake Connection

In [ ]:
# Automatically get the active Snowflake session
session = get_active_session()

# Display session details
details = {
    "Database": session.get_current_database(),
    "Schema": session.get_current_schema(),
    "Role": session.get_current_role(),
    "Warehouse": session.get_current_warehouse()
}

for key, value in details.items():
    print(f"{key}: {value}")

### Helper Functions: Data Generators

In [ ]:
fake = Faker()

def generate_customers(num):
    customers = []
    for i in range(1, num + 1):
        customers.append({
            'customer_id': i,
            'first_name': fake.first_name(),
            'last_name': fake.last_name(),
            'email': fake.email(),
            'phone_number': fake.phone_number(),
            'birth_date': fake.date_of_birth(minimum_age=18, maximum_age=80),
            'country': fake.country(),
            'city': fake.city(),
            'gender': random.choice(['M', 'F', 'Other']),
        })
    return pd.DataFrame(customers)

def generate_stores(num):
    stores = []
    for i in range(1, num + 1):
        stores.append({
            'store_id': i,
            'store_name': fake.company(),
            'region': fake.state(),
            'country': fake.country(),
            'store_type': random.choice(['Urban', 'Suburban', 'Rural']),
        })
    return pd.DataFrame(stores)

def generate_products(num):
    products = []
    for i in range(1, num + 1):
        products.append({
            'product_id': i,
            'product_name': fake.word(),
            'category': random.choice(['Electronics', 'Clothing', 'Groceries', 'Home Appliances', 'Toys']),
            'brand': fake.company(),
            'price': round(random.uniform(5.0, 500.0), 2),
        })
    return pd.DataFrame(products)

def generate_time_dimension(years_back):
    today = pd.Timestamp.today()
    start_date = today - pd.DateOffset(years=years_back)
    dates = pd.date_range(start=start_date, end=today)
    time_dim = []
    for i, date in enumerate(dates):
        time_dim.append({
            'date_id': i + 1,
            'date': date.date(),
            'year': date.year,
            'month': date.month,
            'day': date.day,
            'weekday': date.strftime('%A'),
            'quarter': (date.month - 1) // 3 + 1,
            'is_weekend': date.weekday() >= 5,
        })
    return pd.DataFrame(time_dim)

def generate_sales_fact(num_sales, customers, stores, products, time_dim):
    # Convert columns to NumPy arrays for efficient sampling
    date_ids = time_dim['DATE_ID'].to_numpy()
    customer_ids = customers['CUSTOMER_ID'].to_numpy()
    store_ids = stores['STORE_ID'].to_numpy()
    product_ids = products['PRODUCT_ID'].to_numpy()

    # Pre-sample all IDs
    sampled_customer_ids = np.random.choice(customer_ids, size=num_sales, replace=True)
    sampled_store_ids = np.random.choice(store_ids, size=num_sales, replace=True)
    sampled_product_ids = np.random.choice(product_ids, size=num_sales, replace=True)
    sampled_date_ids = np.random.choice(date_ids, size=num_sales, replace=True)
    sampled_quantities = np.random.randint(1, 11, size=num_sales)  # Quantities between 1 and 10
    sampled_amounts = np.random.uniform(10.0, 1000.0, size=num_sales).round(2)  # Random amounts

    # Create the sales fact DataFrame
    sales_fact = pd.DataFrame({
        'sale_id': [fake.uuid4() for _ in range(num_sales)],
        'date_id': sampled_date_ids,
        'customer_id': sampled_customer_ids,
        'store_id': sampled_store_ids,
        'product_id': sampled_product_ids,
        'quantity': sampled_quantities,
        'total_amount': sampled_amounts
    })
    return sales_fact

### Create Snowflake Tables and Write Data

In [ ]:
# Automatically get the active Snowflake session
session = get_active_session()

num_customers = 1000

# Generate data
customers = generate_customers(num_customers)

# Convert columns to uppercase
customers.columns = customers.columns.str.upper()

# Write directly to Snowflake using Modin
session.write_pandas(customers, "CUSTOMERS", auto_create_table=True, overwrite=True)

In [ ]:
# Automatically get the active Snowflake session
session = get_active_session()

num_stores = 100

# Generate data
stores = generate_stores(num_stores)

# Convert columns to uppercase
stores.columns = stores.columns.str.upper()

# Write directly to Snowflake using Modin
session.write_pandas(stores, "STORES", auto_create_table=True, overwrite=True)

In [ ]:
# Automatically get the active Snowflake session
session = get_active_session()

num_products = 500

# Generate data
products = generate_products(num_products)

# Convert columns to uppercase
products.columns = products.columns.str.upper()

# Write directly to Snowflake using Modin
session.write_pandas(products, "PRODUCTS", auto_create_table=True, overwrite=True)

In [ ]:
# Automatically get the active Snowflake session
session = get_active_session()

years_back = 5

# Generate data
time_dim = generate_time_dimension(years_back)

# Convert columns to uppercase
time_dim.columns = time_dim.columns.str.upper()

# Write directly to Snowflake using Modin
session.write_pandas(time_dim, "TIME_DIMENSION", auto_create_table=True, overwrite=True)

In [ ]:
# Automatically get the active Snowflake session
session = get_active_session()

num_sales = 200000

# Generate data
sales_fact = generate_sales_fact(num_sales, customers, stores, products, time_dim)

# Convert columns to uppercase
sales_fact.columns = sales_fact.columns.str.upper()

# Write directly to Snowflake using Modin
session.write_pandas(sales_fact, "SALES_FACT", auto_create_table=True, overwrite=True)